## Imports

In [1]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import cv2
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sympy import *
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
#from utils.plotter import plot_SVM_DecisionBoundary
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, log_loss, f1_score
from pandas.api.types import CategoricalDtype
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

## Load csvs

In [2]:
path_counts_ran = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/random_per_user_emoji_counts.csv'
path_users_ran = '/home/stelios/Desktop/Honours Project/Samples/user_data/ran.csv'
counts_df_ran = pd.read_csv(path_counts_ran,index_col=None, header=0, low_memory = False)
users_df_ran = pd.read_csv(path_users_ran,index_col=None, header=0, low_memory = False)

path_counts_lon = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/london_per_user_emoji_counts.csv'
path_users_lon = '/home/stelios/Desktop/Honours Project/Samples/user_data/lon.csv'
counts_df_lon = pd.read_csv(path_counts_lon,index_col=None, header=0, low_memory = False)
users_df_lon = pd.read_csv(path_users_lon,index_col=None, header=0, low_memory = False)

path_counts_joh = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/joh_per_user_emoji_counts.csv'
path_users_joh = '/home/stelios/Desktop/Honours Project/Samples/user_data/joh.csv'
counts_df_joh = pd.read_csv(path_counts_joh,index_col=None, header=0, low_memory = False)
users_df_joh = pd.read_csv(path_users_joh,index_col=None, header=0, low_memory = False)

path_counts_nyc = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/nyc_per_user_emoji_counts.csv'
path_users_nyc = '/home/stelios/Desktop/Honours Project/Samples/user_data/nyc.csv'
counts_df_nyc = pd.read_csv(path_counts_nyc,index_col=None, header=0, low_memory = False)
users_df_nyc = pd.read_csv(path_users_nyc,index_col=None, header=0, low_memory = False)

In [3]:
# Append counts from all Locations
counts = counts_df_ran.append(counts_df_lon, sort=False,ignore_index=True).append(counts_df_joh, sort=False,ignore_index=True).append(counts_df_nyc, sort=False,ignore_index=True)
print(counts.shape)

# Append users from all Locations
users = users_df_ran.append(users_df_lon, sort=False,ignore_index=True).append(users_df_joh, sort=False,ignore_index=True).append(users_df_nyc, sort=False,ignore_index=True)
print(users.shape)

# Fill NaN with 0.0
counts = counts.fillna(0.0)

(40000, 2579)
(40000, 28)


In [4]:
# Preview counts
counts.describe()

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,🧗🏻‍♂️,👩🏾‍🚒,👩🏿‍🎨,👨🏿‍✈️,👰🏿,⛹🏿‍♂️,🤹🏽‍♂️,🧜🏿‍♂️,🇵🇲,🇬🇺
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.00000,40000.0000,40000.0000,40000.000000,...,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.0000,40000.0000,40000.000000,40000.000000,40000.00000
mean,0.000025,0.000050,0.000225,0.000300,0.000225,0.000075,0.00020,0.0001,0.0001,0.000050,...,0.000050,0.000075,0.000100,0.000100,0.000125,0.0002,0.0002,0.000225,0.000075,0.00005
std,0.005000,0.007071,0.035707,0.031622,0.022912,0.011180,0.01581,0.0100,0.0100,0.007071,...,0.007071,0.015000,0.015811,0.015811,0.018028,0.0400,0.0400,0.045000,0.015000,0.01000
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.00000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.00000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.00000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.00000
max,1.000000,1.000000,7.000000,5.000000,3.000000,2.000000,2.00000,1.0000,1.0000,1.000000,...,1.000000,3.000000,3.000000,3.000000,3.000000,8.0000,8.0000,9.000000,3.000000,2.00000


In [33]:
(1/counts.loc[3].rank()).min()

0.00078125

## Reciprocal Rank Feuture representation

In [5]:
# RR Feauture representation
for index, row in counts.iterrows():
    counts.loc[index] = 1/counts.loc[index].rank()
counts.describe()

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,🧗🏻‍♂️,👩🏾‍🚒,👩🏿‍🎨,👨🏿‍✈️,👰🏿,⛹🏿‍♂️,🤹🏽‍♂️,🧜🏿‍♂️,🇵🇲,🇬🇺
count,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,...,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
mean,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,...,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780
std,0.000009,0.000009,0.000009,0.000010,0.000010,0.000009,0.000010,0.000009,0.000009,0.000009,...,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009
min,0.000405,0.000396,0.000388,0.000391,0.000394,0.000395,0.000396,0.000392,0.000392,0.000392,...,0.000396,0.000392,0.000389,0.000390,0.000388,0.000388,0.000390,0.000388,0.000388,0.000395
25%,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,...,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775,0.000775
50%,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,...,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777
75%,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782,...,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782,0.000782
max,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950,...,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950,0.000950


## Add gender labels

In [6]:
# Add gender column
gender_counts_df = counts 
gender_counts_df['gender'] = users['gender']
gender_counts_df = gender_counts_df.dropna()

In [17]:
# Drop users that haven't used emojis
min_value_to_drop = gender_counts_df.mean(axis=1).min()
gender_counts_df = gender_counts_df[(gender_counts_df.mean(axis=1)!= min_value_to_drop)]
gender_counts_df

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,👩🏾‍🚒,👩🏿‍🎨,👨🏿‍✈️,👰🏿,⛹🏿‍♂️,🤹🏽‍♂️,🧜🏿‍♂️,🇵🇲,🇬🇺,gender
3,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,...,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,female
4,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,...,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,female
9,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,...,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,female
12,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,...,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,female
14,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,...,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,male
17,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,...,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,male
19,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,...,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,male
21,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,...,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,female
23,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,...,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,female
29,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,...,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,female


In [18]:
# Count males and Females
gender_counts_df['gender']
males_count = 0
females_count = 0

for i in gender_counts_df['gender']:
    if i == 'female':
        females_count = females_count+1
    if i == 'male':
        males_count = males_count+1
print("Males: {}".format(males_count))
print("Females: {}".format(females_count))

Males: 7521
Females: 8484


In [19]:
# Female -> 0
# Male -> 1
gender_counts_df = gender_counts_df.replace('female', 0)
gender_counts_df = gender_counts_df.replace('male', 1)
gender_counts_df

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,👩🏾‍🚒,👩🏿‍🎨,👨🏿‍✈️,👰🏿,⛹🏿‍♂️,🤹🏽‍♂️,🧜🏿‍♂️,🇵🇲,🇬🇺,gender
3,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,...,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0.000781,0
4,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,...,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0.000796,0
9,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,...,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0.000793,0
12,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,...,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0
14,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,...,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,0.000778,1
17,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,...,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,0.000779,1
19,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,...,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,0.000777,1
21,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,...,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0.000780,0
23,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,...,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0.000786,0
29,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,...,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0.000776,0


## Split into test and train

In [20]:
X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(gender_counts_df.drop(columns=['gender']), gender_counts_df['gender'], test_size=0.2, random_state=42)

## SVM with Chi-squared kernel Classification

In [26]:
svm_chi = SVC(kernel=chi2_kernel, probability=True).fit(X_train_gender, y_train_gender)

# Train Data
pred_prob_chi = svm_chi.predict_proba(X_train_gender)
logloss_train_chi = log_loss(y_train_gender, pred_prob_chi)
predict_train_chi  = svm_chi.predict(X_train_gender)
accuracy_train_chi = accuracy_score(y_train_gender, predict_train_chi)
#f1_score_score_train_chi = f1_score(y_train_gender, predict_train_chi)

# Test Data
pred_prob_val_chi = svm_chi.predict_proba(X_test_gender)
logloss_val_chi = log_loss(y_test_gender, pred_prob_val_chi)
predict_val_chi  = svm_chi.predict(X_test_gender)
accuracy_val_chi = accuracy_score(y_test_gender, predict_val_chi)
#f1_score_score_val_chi = f1_score(y_test_gender, predict_val_chi)

# Report
print('Log loss with SVM (Chi-squared kernel) on training set: {:.3f}'.format(logloss_train_chi))
print('Log loss with SVM (Chi-squared kernel) on test set: {:.3f}\n'.format(logloss_val_chi))
print('Accuracy score with SVM (Chi-squared kernel) on training set: {:.3f}'.format(accuracy_train_chi))
print('Accuracy score with SVM (Chi-squared kernel) on test set: {:.3f}\n'.format(accuracy_val_chi))
#print('F-score with SVM (Chi-squared kernel) on training set: {:.3f}'.format(f1_score_score_train_chi))
#print('F-score with SVM (Chi-squared kernel) on test set: {:.3f}'.format(f1_score_score_val_chi))

Log loss with SVM (Chi-squared kernel) on training set: 0.578
Log loss with SVM (Chi-squared kernel) on test set: 0.594

Accuracy score with SVM (Chi-squared kernel) on training set: 0.699
Accuracy score with SVM (Chi-squared kernel) on test set: 0.698



## Linear SVM Classification

In [25]:
clf = LinearSVC(random_state=0, tol=1e-5).fit(X_train_gender, y_train_gender)

# Train Data
predict_train_clf  = clf.predict(X_train_gender)
accuracy_train_clf = accuracy_score(y_train_gender, predict_train_clf)
f1_score_score_train_clf = f1_score(y_train_gender, predict_train_clf,average='macro')

# Test Data

predict_val_clf  = clf.predict(X_test_gender)
accuracy_val_clf = accuracy_score(y_test_gender, predict_val_clf)
f1_score_score_val_clf = f1_score(y_test_gender, predict_val_clf, average='macro')

# Report
print('Accuracy score with Linear SVM on train set: {:.3f}'.format(accuracy_train_clf))
print('Accuracy score with Linear SVM on test set: {:.3f}\n'.format(accuracy_val_clf))
print('F-score with Linear SVM on train set: {:.3f}'.format(f1_score_score_train_clf))
print('F-score with Linear SVM on test set: {:.3f}'.format(f1_score_score_val_clf))

## f1 score scores of labels that were not predicted

Accuracy score with Linear SVM on train set: 0.532
Accuracy score with Linear SVM on test set: 0.521

F-score with Linear SVM on train set: 0.347
F-score with Linear SVM on test set: 0.343


/home/stelios/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/stelios/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
